In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import sys
sys.path.append(os.path.dirname(os.getcwd()))

from common import *

In [ ]:
data              = pd.read_csv(join(PATH.input, "train_data.csv"))
sample_submission = pd.read_csv(join(PATH.input, "sample_submission.csv"))
data.info()

In [ ]:
data_with_time = generate_full_timestamp(data)

In [ ]:
data_imp = data_with_time.fillna(method='bfill')

In [ ]:
data_proc = preprocess(data_imp)

In [ ]:
IN_SEQ_LEN  = 2*144
OUT_SEQ_LEN = 2*144
STRIDE      = 144
SHUFFLE     = False
TEST_SIZE   = 0  # 2/200
BATCH_SIZE  = 256

In [ ]:
cols = ['Time', 'TurbID', 'RPM', 'Bspd1', 'Bspd3', 'Bspd2', 'Wspd_cos', 'Wspd', 'Wspd_cube',
       'P_max', 'Pab1', 'Pab2', 'Pab3', 'TSR1', 'TSR2', 'TSR3', 'Prtv','Patv']
data_fin = data_proc[cols]

In [ ]:
train_x1, train_y1, val_x, val_y, test_x = make_train_val_test_data(data_fin, IN_SEQ_LEN, OUT_SEQ_LEN, STRIDE, SHUFFLE, TEST_SIZE)
# train_ds = generate_dataset(train_x, train_y, batch_size=BATCH_SIZE)
# val_ds   = generate_dataset(val_x, val_y, batch_size=BATCH_SIZE)
# test_ds  = generate_dataset(test_x, batch_size=1)

---

In [ ]:
def make_train_data(data, seq_len, n_train):
    """
    Last training set: [n_train-2-seq_len:n_train-2]
    Last test set    : [n_train-2:n_train]
    """
    train_x, train_y = [], []
    for i in tqdm(sorted(pd.unique(data["TurbID"]))):
        tmp_data = data[data["TurbID"] == i]
        for j in range(1, (n_train-2) - (seq_len-1) + 1):  # 2일 test
            # train data ==> seq_len일 단위
            # label data ==> 2일 단위
            train_days = np.arange(j, j+seq_len)
            label_days = np.arange(j+seq_len, j+seq_len+2)
            if (tmp_data.Day.isin(np.arange(j, j+seq_len+2)).sum() / 144) == (seq_len+2):  # 1Day = 10m * 144
                train_tmp  = tmp_data[tmp_data.Day.isin(train_days)].drop(columns=['TurbID', 'Day'])
                label_tmp  = tmp_data[tmp_data.Day.isin(label_days)].drop(columns=['TurbID', 'Day'])
                train_x.append(train_tmp)  # (720, 11) = (Tmstamp * days, n_features)
                train_y.append(label_tmp)  # (288)     = (Tmstamp * days)
    train_x, train_y = np.array(train_x, dtype=np.float32), np.array(train_y, dtype=np.float32)
    return train_x, train_y
def make_val_data(data, seq_len):
    train_days = np.arange(198-seq_len+1, 199)
    label_days = np.arange(199, 201)
    val_x = data[data.Day.isin(train_days)].drop(columns=['TurbID', 'Day'])
    val_y = data[data.Day.isin(label_days)].Patv
    return np.array(val_x, dtype=np.float32), np.array(val_y, dtype=np.float32)

def make_test_data(data, seq_len):
    test_x, test_y = [], []
    for j in range(201, 240 - seq_len):
        test_days = np.arange(j, j+seq_len)
        label_days = np.arange(j+seq_len, j+seq_len+2)
        test_tmp = data[data.Day.isin(test_days)].drop(columns=['TurbID','Day'])
        label_tmp = data[data.Day.isin(label_days)].drop(columns=['TurbID','Day'])
        test_x.append(test_tmp)  # (720, 11) = (Tmstamp * days, n_features)
        test_y.append(label_tmp)  # (288)     = (Tmstamp * days)
    test_x, test_y = np.array(test_x, dtype=np.float32), np.array(test_y, dtype=np.float32)
    return np.array(test_x, dtype=np.float32), np.array(test_y, dtype=np.float32)

In [ ]:
cols = ['Day', 'TurbID', 'RPM', 'Bspd1', 'Bspd3', 'Bspd2', 'Wspd_cos', 'Wspd', 'Wspd_cube',
       'P_max', 'Pab1', 'Pab2', 'Pab3', 'TSR1', 'TSR2', 'TSR3', 'Prtv','Patv']
data_fin = data_proc[cols]

In [ ]:
train_x2, train_y2 = make_train_data(data_fin, 2, 200)

In [ ]:
train_x1.shape, train_x2.shape

In [ ]:
np.all(train_x1 == train_x2)